In [22]:
import pandas as pd
import numpy as np
from pymoo.optimize import minimize
import pickle

from pymoo.util.termination.f_tol import MultiObjectiveSpaceToleranceTermination
from pymoo.visualization.scatter import Scatter
import contact
from notebooks.optimization_problems.constraints import Requirements
from pyreport import PlotUtil
import matplotlib.pyplot as plt

%matplotlib

plt.close('all')

Using matplotlib backend: Qt5Agg


In [23]:
columns = ['strand_name', 'tof',
           'r_a_x', 'r_a_y', 'r_a_z',
           'v_a_x', 'v_a_y', 'v_a_z',
           'r_b_x', 'r_b_y', 'r_b_z',
           'd',
           'r_ab_sff_x', 'r_ab_sff_y', 'r_ab_sff_z']

file_path = "telesat_post_processed.h5"

row_limit = -1

store_500km = pd.HDFStore(file_path)

instances_500km_df = store_500km.select('contact_instances', 'columns = %s' % str(columns), stop = row_limit)

# Sort by pass id
instances_500km_df = instances_500km_df.sort_index(0)

In [24]:
datafiles = {'1600_100': ('down_selection_n_gen_1600_n_pop_100.pkl', False),
             '1600_400': ('down_selection_n_gen_1600_n_pop_400.pkl', False),
             '1600_100_ds': ('down_selection_n_gen_1600_n_pop_100_ds.pkl', True),
             '1600_400_ds': ('down_selection_n_gen_1600_n_pop_400_ds.pkl', True)}

In [25]:
def _process(res, problem, label, downselect = False):

    x_pass = res.X[:, problem.x_indices['pass']].astype('bool')

    if downselect:
        for i in range(x_pass.shape[0]):
            x_pass[i, :] = contact.down_select_passes(x_pass[i, :], problem.O_matrix)

    # Recompute all four objectives
    f_pointing = np.empty_like(res.F[:,0])
    f_energy = np.empty_like(res.F[:,0])
    f_latency = np.empty_like(res.F[:,0])
    f_throughput = np.empty_like(res.F[:,0])

    scale_factors = np.array([1/-1e9, 1, 1/1e3, 1])

    for i in range(len(f_throughput)):
        ff, gg = problem.evaluate_unmasked(res.X[i,:])
        ff = ff * scale_factors
        f_pointing[i] = ff[3] # s
        f_energy[i] = ff[2] # Kilo Joule
        f_latency[i] = ff[1]   # s
        f_throughput[i] = ff[0]   # Gigabit

    # Marked points
    points = []

    # Max throughput point
    ind = np.argmax(f_throughput)
    p = np.array([f_throughput[ind], f_latency[ind], f_energy[ind], f_pointing[ind]])
    points.append({'ind': ind, 'p': p, 'color': 'r', 'marker': '.', 'size': 10, 'label': 'Max T\n(T=%d Gbit, L=%d s, E=%d kJ, P=%d s)' % (f_throughput[ind], f_latency[ind], f_energy[ind], f_pointing[ind])})

    # Pareto fronts multiple
    limits_on = False

    fig = plt.figure(figsize=(3.2*3, 2.4*3))
    axs = fig.subplots(3,3, gridspec_kw={'height_ratios': [1, 1, 1]})
    axs = axs.flatten()

    ax = axs[0]
    ax.grid(True)
    ax.scatter(f_energy, f_throughput, marker='.', s=1)
    for point in points:
        ax.scatter(point['p'][2], point['p'][0],
                   color=point['color'], s=point['size'], marker=point['marker'], label=point['label'])

    ax.set_xlabel("Energy used [kJ]")
    ax.set_ylabel("Throughput [Gbit]")
    ax.set_axisbelow(True)

    ax = axs[1]
    ax.grid(True)
    ax.scatter(f_pointing, f_throughput, marker='.', s=1)
    for point in points:
        ax.scatter(point['p'][3], point['p'][0],
                   color=point['color'], s=point['size'], marker=point['marker'], label=point['label'])

    ax.set_xlabel("Pointing [s]")
    ax.set_ylabel("Throughput [Gbit]")
    ax.set_axisbelow(True)

    ax = axs[2]
    ax.grid(True)
    ax.scatter(f_latency, f_throughput, marker='.', s=1)
    for point in points:
        ax.scatter(point['p'][1], point['p'][0],
                   color=point['color'], s=point['size'], marker=point['marker'], label=point['label'])

    # ax.scatter(p_maxTminP[0], p_maxTminP[1], color='b', s=10)
    # ax.scatter(p_minEminP[0], p_minEminP[1], color='y', s=10)
    ax.set_xlabel("Avg. Latency [s]")
    ax.set_ylabel("Throughput [Gbit]")
    ax.set_axisbelow(True)

    ax = axs[3]
    ax.grid(True)
    ax.scatter(f_energy, f_latency, marker='.', s=1)
    for point in points:
        ax.scatter(point['p'][2], point['p'][1],
                   color=point['color'], s=point['size'], marker=point['marker'], label=point['label'])

    ax.set_xlabel("Energy used [kJ]")
    ax.set_ylabel("Avg. Latency [s]")
    ax.set_axisbelow(True)

    ax = axs[4]
    ax.grid(True)
    ax.scatter(f_pointing, f_latency, marker='.', s=1)
    for point in points:
        ax.scatter(point['p'][3], point['p'][1],
                   color=point['color'], s=point['size'], marker=point['marker'], label=point['label'])

    ax.set_xlabel("Pointing [s]")
    ax.set_ylabel("Avg. Latency [s]")
    ax.set_axisbelow(True)

    ax = axs[5]
    ax.grid(True)
    ax.scatter(f_energy, f_pointing, marker='.', s=1)
    for point in points:
        ax.scatter(point['p'][2], point['p'][3],
                   color=point['color'], s=point['size'], marker=point['marker'], label=point['label'])

    # ax.scatter(p_maxTminP[0], p_maxTminP[1], color='b', s=10)
    # ax.scatter(p_minEminP[0], p_minEminP[1], color='y', s=10)
    ax.set_xlabel("Energy used [kJ]")
    ax.set_ylabel("Pointing [s]")
    ax.set_axisbelow(True)

    axs[6].set_axis_off()
    axs[7].set_axis_off()
    ax = axs[8]
    for point in points:
        ax.scatter(np.nan, np.nan,
                   color=point['color'], s=point['size'], marker=point['marker'], label=point['label'])

    ax.legend(fontsize=8)
    ax.set_axis_off()

    plt.tight_layout()

    # Plot selected passes of max throughput solution
    ind = points[0]['ind']
    pass_ind = np.nonzero(x_pass[ind,:])[0]

    ff, gg = problem.evaluate_unmasked(res.X[ind,:])

    # Plot range
    T_orbit = 5680
    T_sim = T_orbit * 3

    fig = plt.figure(figsize=(3.2, 2.4))

    # Plot all passes
    for i, pass_df in instances_500km_df.groupby(level=0):
        p = pass_df.index[0][0] - 1 # Pass index
        tof = pass_df.tof.values
        d = pass_df.d.values/1000
        line, = plt.plot(tof, d, linewidth=0.1, color='tab:grey')

    # Plot used passes
    for i, pass_df in instances_500km_df.groupby(level=0):
        p = pass_df.index[0][0] - 1 # Pass index
        tof = pass_df.tof
        d = pass_df.d/1000
        if p in pass_ind:
            line, = plt.plot(tof, d, linewidth=0.5, color='tab:red')

    plt.xlabel('Time of flight [s]')
    plt.ylabel('Range [km]')
    plt.xlim((0, T_sim))

    plt.grid()
    PlotUtil.apply_report_formatting()
    fig.set_size_inches(3.2*2, 2.4, forward=True)

    plt.tight_layout()

    plt.savefig('D:/git/thesis_report_ae/figures/down_select_%s.pdf' % label)

    print(points[0]['label'])
    #plt.savefig('D:/git/thesis_report_ae/figures/relay_optimization/telesat_range_selection_%s_%d.svg' % (setting['label'], res.algorithm.n_gen))

for k, v in datafiles.items():
    problem, setting, res = pickle.load(open(v[0], 'rb'))
    _process(res, problem, k, downselect=v[1])


Max T
(T=3638 Gbit, L=1145 s, E=109 kJ, P=7622 s)
Max T
(T=4503 Gbit, L=1550 s, E=193 kJ, P=5793 s)
Max T
(T=8614 Gbit, L=393 s, E=388 kJ, P=12710 s)
Max T
(T=9508 Gbit, L=338 s, E=458 kJ, P=12544 s)
